In [1]:
!pip install --upgrade git+https://github.com/flairNLP/flair.git

  Cloning https://github.com/flairNLP/flair.git to /tmp/pip-req-build-cbd67jg1
  Running command git clone -q https://github.com/flairNLP/flair.git /tmp/pip-req-build-cbd67jg1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 665kB 21.2MB/s 
     |████████████████████████████████| 798kB 56.3MB/s 
     |████████████████████████████████| 256kB 51.4MB/s 
     |████████████████████████████████| 983kB 48.8MB/s 
     |████████████████████████████████| 3.8MB 54.4MB/s 
     |████████████████████████████████| 890kB 43.5MB/s 
     |████████████████████████████████| 1.1MB 53.8MB/s 
  Created wheel for flair: filename=flair-0.5-cp36-none-any.whl size=148939 sha256=aeb7ba1310603af19d4551b9aeb8b7f533b4b5dcb083c77651134f0f96208736
  Stored in directory: /tmp/pip-ephem-wheel-cache-1fdfd1__/wheels/84/82/73/d2b3b59b7be74ea05f2c6d64132efe27df52daffb47d1dc7bb
Successfully built flair
  Creat

In [0]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings, DocumentRNNEmbeddings, BertEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from flair.datasets import CSVClassificationCorpus
from flair.data import Corpus
import pandas as pd

from flair.data import Sentence

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data_folder = "./drive/My Drive/Colab Notebooks/capstone/data/"

### First Stage (Train on benchmark dataset)

In [0]:
benchmark = pd.read_csv(data_folder + "combined_benchmark.csv")

In [6]:
benchmark = benchmark[['label', 'text']]
benchmark.head()

,label,text
0,0,Why not subscribe to the magazine ?
1,-1,"Tornio Works employs 2,300 of whom more than 1..."
2,1,"The move is aimed at boosting sales , cost-eff..."
3,0,"As a result of the merger , the largest profes..."
4,-1,18 March 2010 A leakage in the gypsum pond was...


#### Create train, dev and test set

In [0]:
benchmark = benchmark.sample(frac=1, random_state=42)


benchmark.iloc[0:int(len(benchmark)*0.8)].to_csv(data_folder + 'train.csv', sep=',', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.8):int(len(benchmark)*0.9)].to_csv(data_folder + 'test.csv', sep=',', index = False, header = False)
benchmark.iloc[int(len(benchmark)*0.9):].to_csv(data_folder + 'dev.csv', sep=',', index = False, header = False)

In [8]:
train_df = pd.read_csv(data_folder + "train.csv", header = None)
train_df.head()

,0,1
0,1,About Elcoteq Elcoteq SE is a leading electron...
1,-1,U.S. goods trade deficit deteriorates; factory...
2,0,Product coverage : baked goods ; biscuits ; br...
3,0,Tyrv+Æinen is of the opinion that the airline ...
4,-1,Wall St. Week Ahead: U.S. stock reign may not ...


#### Build corpus

In [9]:
# corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_folder), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')
column_name_map = {1: "text", 0: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(data_folder,
                                         column_name_map,
                                         skip_header=False, #no header in kaggle data
                                         delimiter=',',    # comma separated rows
)

2020-05-28 01:48:25,270 Reading data from drive/My Drive/Colab Notebooks/capstone/data
2020-05-28 01:48:25,273 Train: drive/My Drive/Colab Notebooks/capstone/data/train.csv
2020-05-28 01:48:25,278 Dev: drive/My Drive/Colab Notebooks/capstone/data/dev.csv
2020-05-28 01:48:25,279 Test: drive/My Drive/Colab Notebooks/capstone/data/test.csv


#### Create word embeddings

In [10]:
word_embeddings = [BertEmbeddings(), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.
  """Entry point for launching an IPython kernel.



2020-05-28 01:48:47,646 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmpdi50xosv


100%|██████████| 19689779/19689779 [00:04<00:00, 4626158.25B/s]

2020-05-28 01:48:53,224 copying /tmp/tmpdi50xosv to cache at /root/.flair/embeddings/lm-news-english-forward-1024-v0.2rc.pt
2020-05-28 01:48:53,248 removing temp file /tmp/tmpdi50xosv


2020-05-28 01:49:12,089 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward-1024-v0.2rc.pt not found in cache, downloading to /tmp/tmp7ksl7k1w


100%|██████████| 19689779/19689779 [00:04<00:00, 4730370.01B/s]

2020-05-28 01:49:17,527 copying /tmp/tmp7ksl7k1w to cache at /root/.flair/embeddings/lm-news-english-backward-1024-v0.2rc.pt
2020-05-28 01:49:17,547 removing temp file /tmp/tmp7ksl7k1w


#### First Stage Fine-tuning

In [11]:
document_embeddings = DocumentLSTMEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)
trainer = ModelTrainer(classifier, corpus)
trainer.train(data_folder,
              learning_rate=0.05, 
              max_epochs=15)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated method __init__. (The functionality of this class is moved to 'DocumentRNNEmbeddings') -- Deprecated since version 0.4.
  """Entry point for launching an IPython kernel.


2020-05-28 01:50:22,214 Computing label dictionary. Progress:


100%|██████████| 1314/1314 [00:01<00:00, 959.84it/s]

2020-05-28 01:50:23,880 [b'1', b'-1', b'0']


2020-05-28 01:50:24,689 ----------------------------------------------------------------------------------------------------
2020-05-28 01:50:24,695 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [0.9019950032234192,
  1.1538550853729248,
  0.8150318264961243,
  0.7469678521156311,
  0.9261205792427063,
  1.0268752574920654,
  0.9452976584434509,
  0.762126624584198,
  0.8327164053916931,
  0.8165239691734314,
  0.930494487285614,
  0.8186808824539185,
  0.901214599609375,
  0.8072444200515747,
  0.8259391784667969],
 'dev_score_history': [0.7006802721088435,
  0.7142857142857143,
  0.782312925170068,
  0.7687074829931972,
  0.691609977324263,
  0.6961451247165533,
  0.7505668934240363,
  0.7732426303854876,
  0.7551020408163265,
  0.7777777777777778,
  0.7732426303854876,
  0.7913832199546486,
  0.782312925170068,
  0.800453514739229,
  0.7913832199546486],
 'test_score': 0.7534246575342466,
 'train_loss_history': [1.0146753707447567,
  0.8344538695103413,
  0.7572574841009604,
  0.6848842445257548,
  0.7039970771686451,
  0.6330873716521908,
  0.6067641562706715,
  0.4849680855467513,
  0.44107394927256816,
  0.4217929240014102,
  0.39998088172964147,
  0

### Second Stage (train on hand annotated datasets)

#### Build corpus

In [28]:
new_data_folder = "./drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled"
new_column_name_map = {5: "text", 4: "label_topic"}

corpus: Corpus = CSVClassificationCorpus(new_data_folder,
                                         new_column_name_map,
                                         skip_header=True, #yes header in 
                                         delimiter=',',    # comma separated rows
)

2020-05-28 02:53:45,186 Reading data from drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled
2020-05-28 02:53:45,187 Train: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/train.csv
2020-05-28 02:53:45,189 Dev: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/dev.csv
2020-05-28 02:53:45,190 Test: drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/test.csv


#### Second Stage fine-tuning

In [30]:
# benchmark_embeddings = FlairEmbeddings(data_folder + 'best-model.pt')
#data_folder = wherever the best-model.pt is put

benchmark_classifier = TextClassifier.load(data_folder + 'best-model.pt')

2020-05-28 02:53:56,331 loading file ./drive/My Drive/Colab Notebooks/capstone/data/best-model.pt


In [31]:
trainer = ModelTrainer(benchmark_classifier, corpus)
trainer.train(new_data_folder, 
              learning_rate=0.05,
              max_epochs=20)


2020-05-28 02:54:08,202 ----------------------------------------------------------------------------------------------------
2020-05-28 02:54:08,208 Model: "TextClassifier(
  (document_embeddings): DocumentLSTMEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): BertEmbeddings(
        (model): BertModel(
          (embeddings): BertEmbeddings(
            (word_embeddings): Embedding(30522, 768, padding_idx=0)
            (position_embeddings): Embedding(512, 768)
            (token_type_embeddings): Embedding(2, 768)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (encoder): BertEncoder(
            (layer): ModuleList(
              (0): BertLayer(
                (attention): BertAttention(
                  (self): BertSelfAttention(
                    (query): Linear(in_features=768, out_features=768, bias=True)
                    (key): Linear(in_feat

{'dev_loss_history': [1.811440110206604,
  2.0882046222686768,
  1.1117914915084839,
  0.9923803210258484,
  1.1513042449951172,
  1.442038893699646,
  1.5963996648788452,
  1.0166499614715576,
  1.0162858963012695,
  1.1404460668563843,
  1.3253134489059448,
  1.1137051582336426,
  1.0922951698303223,
  1.110708475112915,
  1.1028305292129517,
  1.1079765558242798,
  1.1085872650146484,
  1.113206386566162,
  1.094724416732788,
  1.0950584411621094],
 'dev_score_history': [0.52,
  0.5466666666666666,
  0.7866666666666666,
  0.7333333333333333,
  0.6533333333333333,
  0.6533333333333333,
  0.68,
  0.76,
  0.76,
  0.7866666666666666,
  0.6533333333333333,
  0.6533333333333333,
  0.68,
  0.6533333333333333,
  0.7066666666666667,
  0.68,
  0.68,
  0.7066666666666667,
  0.7333333333333333,
  0.7333333333333333],
 'test_score': 0.7866666666666666,
 'train_loss_history': [1.7137648860613506,
  1.4263707101345062,
  0.9512742360432943,
  0.8207606077194214,
  0.6268080621957779,
  0.389255195

In [32]:
mortgage_classifier = TextClassifier.load(new_data_folder + '/best-model.pt')

2020-05-28 03:20:44,817 loading file ./drive/My Drive/Colab Notebooks/capstone/data/phase_2_mortgage_rate_oversampled/best-model.pt


In [0]:
pred_df = pd.read_csv(data_folder + "mortgage_rates_CBC_article_to_predict.csv")

In [54]:
pred_df.head()

,Unnamed: 0,source,title_desc,publishedAt
0,0,CBC,Mortgage arrears rate could spike to double wh...,2020-05-14
1,1,CBC,Interest rates are plunging — so why aren't mo...,2020-04-04
2,2,CBC,Why worries about the coronavirus are pushing ...,2020-01-29
3,3,CBC,U.S. Fed chair rules out negative interest rat...,2020-05-13
4,4,CBC,Mortgages in arrears in Alberta hit highest ra...,2019-08-14


In [55]:
pred_list = []
conf_list = []

for index, row in pred_df.iterrows():
  article_text = row["title_desc"]
  article_sentence = Sentence(article_text)
  mortgage_classifier.predict(article_sentence)
  pred = article_sentence.labels[0].value
  print(pred)
  pred_list.append(pred)

  conf_score = article_sentence.labels[0].score
  print(conf_score)
  conf_list.append(conf_score)

  print('----')


0
0.5833262801170349
----
0
0.8457203507423401
----
-1
0.3939833641052246
----
-1
0.7230633497238159
----
-1
0.7956459522247314
----
-1
0.5243971347808838
----
0
0.8241885900497437
----
-1
0.5161066651344299
----
0
0.9054300785064697
----
0
0.5838729739189148
----
-1
0.9202001690864563
----
-1
0.7700586915016174
----
-1
0.7929283380508423
----
-1
0.5186101198196411
----
-1
0.7283235192298889
----
0
0.7462179660797119
----
0
0.9179391264915466
----
-1
0.723307728767395
----
-1
0.723307728767395
----
0
0.9848251938819885
----
-1
0.6162759065628052
----
0
0.8945677876472473
----
-1
0.6524528861045837
----
-1
0.8799123167991638
----
-1
0.61030113697052
----
-1
0.61030113697052
----
0
0.9861264824867249
----
0
0.987683117389679
----
-1
0.6217166185379028
----
0
0.4838629364967346
----
0
0.8227150440216064
----
-1
0.5220191478729248
----
0
0.8764476776123047
----
0
0.9738495945930481
----
0
0.7048612236976624
----
0
0.9464749097824097
----
-1
0.6920495629310608
----
0
0.9608901143074036
----

In [56]:
print(len(conf_list))
print(len(pred_list))

330
330


In [0]:

pred_df["pred_label"] = pd.Series(pred_list)
pred_df["confidence"] = pd.Series(conf_list)  

In [58]:
pred_df.head()

,Unnamed: 0,source,title_desc,publishedAt,pred_label,confidence
0,0,CBC,Mortgage arrears rate could spike to double wh...,2020-05-14,0,0.583326
1,1,CBC,Interest rates are plunging — so why aren't mo...,2020-04-04,0,0.845720
2,2,CBC,Why worries about the coronavirus are pushing ...,2020-01-29,-1,0.393983
3,3,CBC,U.S. Fed chair rules out negative interest rat...,2020-05-13,-1,0.723063
4,4,CBC,Mortgages in arrears in Alberta hit highest ra...,2019-08-14,-1,0.795646


In [59]:
for index, row in pred_df.iterrows():
  print(row["title_desc"])
  print(row["pred_label"])

Mortgage arrears rate could spike to double what it was in 2009, Bank of Canada says. Central bank says number of people falling behind on mortgages could almost quadruple
0
Interest rates are plunging — so why aren't mortgage rates?. Bond yields and central bank rates have never been lower, but not all the savings are filtering down
0
Why worries about the coronavirus are pushing mortgage rates down. Fear has pushed investors to buy up bonds, which is causing cheaper borrowing for home buyers
-1
U.S. Fed chair rules out negative interest rates even as Trump trumpets them. U.S. president goes negative on Jerome Powell for rejection of below-zero interest rates
-1
Mortgages in arrears in Alberta hit highest rate since 2013. Rate seen as a 'lagging indicator' of the economy, driven largely by unemployment
-1
'Pretty cheap money': Canadian mortgage rates falling to their lowest level in 2 years. Fixed and variable loans have gotten cheaper because costs for lenders are down too
-1
Housing

In [61]:
out_filepath = data_folder + "/unannotated_mortgagerate_CBC_predictions.csv"
print(out_filepath)

./drive/My Drive/Colab Notebooks/capstone/data//unannotated_mortgagerate_CBC_predictions.csv


In [0]:
pred_df.to_csv(out_filepath,index=False)

In [63]:
mort_test_df = pd.read_csv(data_folder + "/test.csv", header=None)

mort_test_df.head()
  

,0,1
0,-1,Finnish metal products company Componenta Oyj ...
1,1,Incap and Lankapaja aim to enter into actual a...
2,-1,Turkish stocks tumble as crackdown on lira spe...
3,1,"Summer Winds Down, and Big Tech Is Called Befo..."
4,-1,Here's why the slump in semiconductor stocks m...


In [65]:
correct = 0
severe_incorrect = 0
for i in range(len(corpus.test)):
  print(i)

  
  #get predictions
  #print(gdp_test_df['title_desc'].iloc[i])
  #print(mort_test_df.iloc[i,1])
  sentence = Sentence(mort_test_df.iloc[i,1])
  mortgage_classifier.predict(sentence)
  #print(sentence.labels)
  # #get gold label
  # print(corpus.test[i].labels)
  # #calculate correct guesses
  if sentence.labels[0].value == corpus.test[i].labels[0].value:
    correct += 1
  else:

    print("INCORRECT CLASSIFICATION: ")
    print("TEXT: ", sentence)
    print("TRUE LABEL: ", corpus.test[i].labels[0].value)
    print("PRED: ",sentence.labels[0].value)
    
    if corpus.test[i].labels[0].value != "0" and sentence.labels[0].value != "0":
      severe_incorrect += 1


  print("----")


0
----
1
----
2
----
3
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Summer Winds Down, and Big Tech Is Called Before Congress. The newest jobs numbers are expected to show healthy gains, and Detroit automakers will report sales for August."   [− Tokens: 28  − Sentence-Labels: {'class': [0 (0.5657)]}]
TRUE LABEL:  -1
PRED:  0
----
4
----
5
----
6
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Many listed stocks currently undervalued. More than half of listed stocks here are trading at prices below "book value" . suited for valuing capital-intensive companies or financial businesses with a lot of ."   [− Tokens: 33  − Sentence-Labels: {'class': [1 (0.4755)]}]
TRUE LABEL:  0
PRED:  1
----
7
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "Revenue grew 1 percent to euro742 .2 million US$ 964 million from euro735 million ."   [− Tokens: 15  − Sentence-Labels: {'class': [0 (0.8079)]}]
TRUE LABEL:  1
PRED:  0
----
8
INCORRECT CLASSIFICATION: 
TEXT:  Sentence: "The Point Village , designed by Scott 

In [66]:

print("CORRECT: ",correct/len(corpus.test))
print("INCORRECT: ",1 - correct/len(corpus.test))
print("OPPOSITE PREDICTIONS (SEVERELY INCORRECT): ",severe_incorrect/len(corpus.test))

CORRECT:  0.68
INCORRECT:  0.31999999999999995
OPPOSITE PREDICTIONS (SEVERELY INCORRECT):  0.0
